TEST VERSION BP 230107

<h1>RSoXS plan simulator</h1>
this is a full stack of the SST scan system, from the loader through to the low level acquisition code, all simulated here
run all the hidden cells first!

In [1]:
%load_ext autoreload
%autoreload 2
from rsoxs_scans.spreadsheets import load_samplesxlsx, save_samplesxlsx
from rsoxs_scans.acquisition import dryrun_bar, get_acq_details

In [2]:
# test your .xlsx file here
# load the example bar
bar = load_samplesxlsx(r"Sample_Bar_template_v2023_1.xlsx")

Reading sheet Bar
Reading sheet Acquisitions


/srv/conda/envs/notebook/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
save_samplesxlsx(bar,'test2.xlsx') # test writing to a temporary file

In [4]:
bar2 = load_samplesxlsx("test2.xlsx") # test loading from a temporary file

Reading sheet Bar
Reading sheet Acquisitions


In [ ]:
# dry run it
outputs = dryrun_bar(bar,['apriority'],group='all')

In [ ]:
# Get more details on a given acquisition
get_acq_details(40,outputs)

In [ ]:
# get the set of all actions
actions = set()
for acq in outputs:
    actions.add(acq['action'])
print(actions)

In [ ]:
arguments = {}
for action in actions:
    keys = set()
    for acq in outputs:
        if acq['action'] == action:
            if 'kwargs' in acq:
                for kwarg in acq['kwargs']:
                    keys.add(kwarg)
    arguments[action]=keys
    print("{:<18} : {}".format(action,keys))

In [ ]:
def get_queue(acq,outputs):
    return list(filter(lambda outputs: outputs['acq_index'] == acq, outputs))

for step in get_queue(0,outputs):
            print("-"*50)
            print(f"Step: {step['queue_step']}")
            print("-"*50)
            for key, value in step.items():
                if isinstance(value, dict):
                    print(f"\t{key}", " : ")
                    for key2, value2 in value.items():
                        if isinstance(value2, dict):
                            print(f"\t\t{key2}", " : ")
                            for key3, value3 in value2.items():
                                print(f"\t\t\t{key3}", " : ", value3)
                        else:
                            print(f"\t\t{key2}", " : ", value2)
                else:
                    print(f"\t{key}", " : ", value)

<h1>Examples of edges, ratios, frames

In [ ]:
# totally manual simple scan
from rsoxs_scans.constructor import get_energies
energies = get_energies((1850,1930),100)

In [ ]:
# precise energy list
# you can fool the algorithm if you put 1 as the number of frames it will only take one frame at each of the thresholds, including the last one, so you can sort of treat this as just an energy list
# as a corollary, you can't have a total number of frames less than the thresholds that you give it in edges (it will at a minimum always take the threshold values)
energies = get_energies((250,250,250,340,340,341,280,281.45,500,500),1)

In [ ]:
# use a default
energies = get_energies('Nitrogen')

In [ ]:
# use a default with a different name
energies = get_energies('c')

In [ ]:
# use a different built in interval set
energies = get_energies('c','full','carbon nonaromatic')

In [ ]:
# get a very short version of this scan
energies = get_energies('c','very short','carbon nonaromatic')

In [ ]:
# I don't have time for this, I only want ~10 energies
energies = get_energies('c',10,'carbon nonaromatic')

In [ ]:
# also, I made single entry energies also work
energies = get_energies(280)

In [ ]:
# also option to have frames as a list of exact numbers needed in each region
energies = get_energies('c',[2,5,10,40,5,2])

In [ ]:
# putting in zero effectively removes that region
energies = get_energies('c',[2,5,10,0,5,2])

In [ ]:
from rsoxs_scans.constructor import construct_exposure_times
times,_ = construct_exposure_times(get_energies([270,280,290,400],500,[2,1,10]),1)
len(times)

In [ ]:
from rsoxs_scans.constructor import get_nexafs_scan_params
get_nexafs_scan_params('n','normal')

<h1>Example of complex Exposure times

In [ ]:
construct_exposure_times(get_energies((1850,1930),100),[2,("between",1870,1900.0),4,("greater_than",1920.0),1],80)